<a href="https://colab.research.google.com/github/ShashwotP/GroupE_G12/blob/main/CUDAweek8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Dec 24 09:07:23 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
%%writefile firstC.cu
#include <stdio.h>
int main(void)
{
  printf("Hello world from CPU!\n");
}

Writing firstC.cu


In [ ]:
!nvcc firstC.cu -o 1

In [ ]:
!./1

Hello world from CPU!


In [ ]:
%%writefile hello.cu
#include <stdio.h>
__global__ void hello(void)
{
  printf("Hello world from CPU!\n");

}
int main(int argc, char **argv)
{
  printf("CPU: Hello!:\n");
  hello<<<1,10>>>();
  cudaDeviceReset();
  return 0;
}

Overwriting hello.cu


In [ ]:
!nvcc hello.cu -o hello

In [ ]:
!./hello

CPU: Hello!:
Hello world from CPU!
Hello world from CPU!
Hello world from CPU!
Hello world from CPU!
Hello world from CPU!
Hello world from CPU!
Hello world from CPU!
Hello world from CPU!
Hello world from CPU!
Hello world from CPU!


In [ ]:
%%writefile dq00.cu
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <cuda_runtime_api.h>

/*****************************************************************************
 * This program checks if it is being run on a computer with a CUDA compatible
 * GPU, i.e. a modern nVidia GPU. If this program reports there are no
 * devices then no other programs in this set are going to work on the machine
 * being run on.
 *
 * Compile with:
 *   nvcc -o dq00 dq00.cu
 *
 * Dr Kevan Buckley, University of Wolverhampton, 2018
 ****************************************************************************/
int main() {
  int device_count;
  cudaError_t error_id = cudaGetDeviceCount(&device_count);

  if (error_id != 0) {
    fprintf(stderr, "cudaGetDeviceCount returned %d\n-> %s\n",
           (int)error_id, cudaGetErrorString(error_id));
    exit(1);
  }

  if (device_count == 0) {
    fprintf(stderr, "There are no available device(s) that support CUDA\n");
  } else {
    printf("Detected %d CUDA capable device(s)\n", device_count);
  }

  return 0;
}

Writing dq00.cu


In [ ]:
!nvcc -o dq00 dq00.cu

In [ ]:
!./dq00

Detected 1 CUDA capable device(s)


In [ ]:
%%writefile dq01.cu
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <cuda_runtime_api.h>

/*****************************************************************************
 * This program introduces using cudaGetDeviceProperties to find about the
 * available GPUs. Information is returned in a cudaDeviceProp data structure
 * which can be found, with a limited amount of documentation in
 * cuda_runtim_api.h. A lot of the fields are self explanatory so the struct
 * declaration is copied below for convenience. The next few programs in this
 * set demonstrate using this information. When you have tried them you could
 * look at investigating some of the other available fields and by reading
 * about what they mean gain a greater understanding of CUDA and GPU
 * architecture.
 *
    struct cudaDeviceProp {
      char name[256];
      size_t totalGlobalMem;
      size_t sharedMemPerBlock;
      int regsPerBlock;
      int warpSize;
      size_t memPitch;
      int maxThreadsPerBlock;
      int maxThreadsDim[3];
      int maxGridSize[3];
      int clockRate;
      size_t totalConstMem;
      int major;
      int minor;
      size_t textureAlignment;
      size_t texturePitchAlignment;
      int deviceOverlap;
      int multiProcessorCount;
      int kernelExecTimeoutEnabled;
      int integrated;
      int canMapHostMemory;
      int computeMode;
      int maxTexture1D;
      int maxTexture1DMipmap;
      int maxTexture1DLinear;
      int maxTexture2D[2];
      int maxTexture2DMipmap[2];
      int maxTexture2DLinear[3];
      int maxTexture2DGather[2];
      int maxTexture3D[3];
      int maxTexture3DAlt[3];
      int maxTextureCubemap;
      int maxTexture1DLayered[2];
      int maxTexture2DLayered[3];
      int maxTextureCubemapLayered[2];
      int maxSurface1D;
      int maxSurface2D[2];
      int maxSurface3D[3];
      int maxSurface1DLayered[2];
      int maxSurface2DLayered[3];
      int maxSurfaceCubemap;
      int maxSurfaceCubemapLayered[2];
      size_t surfaceAlignment;
      int concurrentKernels;
      int ECCEnabled;
      int pciBusID;
      int pciDeviceID;
      int pciDomainID;
      int tccDriver;
      int asyncEngineCount;
      int unifiedAddressing;
      int memoryClockRate;
      int memoryBusWidth;
      int l2CacheSize;
      int maxThreadsPerMultiProcessor;
      int streamPrioritiesSupported;
      int globalL1CacheSupported;
      int localL1CacheSupported;
      size_t sharedMemPerMultiprocessor;
      int regsPerMultiprocessor;
      int managedMemSupported;
      int isMultiGpuBoard;
      int multiGpuBoardGroupID;
      int singleToDoublePrecisionPerfRatio;
      int pageableMemoryAccess;
      int concurrentManagedAccess;
    }
 *
 * Compile with:
 *   nvcc -o dq01 dq01.cu
 *
 * Dr Kevan Buckley, University of Wolverhampton, 2018
 ****************************************************************************/


int main() {

  // cudaSetDevice(0); // only needed if there are multiple GPUs

  cudaDeviceProp device_properties;
  cudaGetDeviceProperties(&device_properties, 0);

  int driver_version, runtime_version;
  cudaDriverGetVersion(&driver_version);
  cudaRuntimeGetVersion(&runtime_version);

  printf("Device 0 is a \"%s\"\n", device_properties.name);
  printf("  Compute capability %d.%d\n", device_properties.major,
                                         device_properties.minor);

  printf("  CUDA Driver version %d.%d\n",
         driver_version/1000, (driver_version%100)/10);

  printf("  CUDA runtime version %d.%d\n",
         runtime_version/1000, (runtime_version%100)/10);


  return 0;
}

Writing dq01.cu


In [ ]:
!nvcc -o dq01 dq01.cu

In [ ]:
!./dq01

Device 0 is a "Tesla T4"
  Compute capability 7.5
  CUDA Driver version 12.2
  CUDA runtime version 12.2


In [ ]:
%%writefile dq02.cu
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <cuda_runtime_api.h>

/*****************************************************************************
 * The most complicated thing this program does is work out how many CUDA
 * cores are available. This is based on the particular architecture found.
 * Documentation for this is difficult to find. The
 * convert_compute_capability_to_cores function used here was copied from the
 * examples that come with CUDA (but other versions exist on the Internet with
 * different copyright notices!). This code really over complicates things so
 * just take it for granted that it works. It will need to be updated as new
 * GPU architectures become available and is known to be out of date at the
 * time this source code set was put together, i.e. the Volta architecture
 * has superceded Pascal (found in GTX1000 series consumer GPUs, such as the
 * GTX1070s in wlv labs)
 *
 * Compile with:
 *   nvcc -o dq02 dq02.cu
 *
 * Dr Kevan Buckley, University of Wolverhampton, 2018
 ****************************************************************************/

// This function which is heavily  based on
// NVIDIA_CUDA-8.0_Samples/common/inc/helper_cuda.h _ConvertSMVer2Cores()
// takes a compute capability (in the form of 2 parameters and returns the
// number of cores per multiprocessor. The algorithm is complicated by
// not all major, minor pairs being valid it might be more readable using a
// sparse array, but to aid the update of this code when a new architecture
// comes out as little as possible has been changed.

int convert_compute_capability_to_cores(int major, int minor) {
    // Defines for GPU Architecture types (using the SM version to determine
    // the # of cores per SM
    typedef struct {
        int SM; // 0xMm (hexidecimal notation)
                // M = SM Major version
                // m = SM minor version
        int Cores;
    } sSMtoCores;

    sSMtoCores nGpuArchCoresPerSM[] = {
      { 0x20, 32 }, // Fermi Generation (SM 2.0) GF100 class
      { 0x21, 48 }, // Fermi Generation (SM 2.1) GF10x class
      { 0x30, 192}, // Kepler Generation (SM 3.0) GK10x class
      { 0x32, 192}, // Kepler Generation (SM 3.2) GK10x class
      { 0x35, 192}, // Kepler Generation (SM 3.5) GK11x class
      { 0x37, 192}, // Kepler Generation (SM 3.7) GK21x class
      { 0x50, 128}, // Maxwell Generation (SM 5.0) GM10x class
      { 0x52, 128}, // Maxwell Generation (SM 5.2) GM20x class
      { 0x53, 128}, // Maxwell Generation (SM 5.3) GM20x class
      { 0x60, 64 }, // Pascal Generation (SM 6.0) GP100 class
      { 0x61, 128}, // Pascal Generation (SM 6.1) GP10x class
      { 0x62, 128}, // Pascal Generation (SM 6.2) GP10x class
      {   -1, -1 }
    };

    int index = 0;

    while (nGpuArchCoresPerSM[index].SM != -1) {
      if (nGpuArchCoresPerSM[index].SM == ((major << 4) + minor)) {
          return nGpuArchCoresPerSM[index].Cores;
      }

      index++;
    }

    // If we don't find the values, we default use the previous one to run properly
    printf("MapSMtoCores for SM %d.%d is undefined. Default to use %d Cores/SM\n",
           major, minor, nGpuArchCoresPerSM[index-1].Cores);
    return nGpuArchCoresPerSM[index-1].Cores;
}


int main() {

  cudaDeviceProp device_properties;
  cudaGetDeviceProperties(&device_properties, 0);

  printf("Device 0 is a \"%s\"\n", device_properties.name);

  printf("  Clock rate = %d MHz\n", device_properties.clockRate/1000);
  printf("  Memory = %lu bytes\n", device_properties.totalGlobalMem);

  int cores_per_multiprocessor =
    convert_compute_capability_to_cores(device_properties.major,
                                        device_properties.minor);
  printf("  Number of multi processors = %d\n",
         device_properties.multiProcessorCount);

  printf("  Cores per multiprocessors = %d\n",
         cores_per_multiprocessor);

  printf("  Total CUDA cores = %d\n",
         cores_per_multiprocessor * device_properties.multiProcessorCount);

  return 0;
}

Writing dq02.cu


In [ ]:
!nvcc -o dq02 dq02.cu

In [ ]:
!./dq02

Device 0 is a "Tesla T4"
  Clock rate = 1590 MHz
  Memory = 15835660288 bytes
MapSMtoCores for SM 7.5 is undefined. Default to use 128 Cores/SM
  Number of multi processors = 40
  Cores per multiprocessors = 128
  Total CUDA cores = 5120


In [ ]:
%%writefile dq03.cu
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <cuda_runtime_api.h>

/*****************************************************************************
 * This program introduces using cudaGetDeviceProperties to find about the
 * available GPUs. This version of the program concentrates on the
 * multithreading capabilities: *

      int maxThreadsPerBlock;
      int maxThreadsDim[3];
      int maxGridSize[3];

 *
 * Compile with:
 *   nvcc -o dq03 dq03.cu
 *
 * Dr Kevan Buckley, University of Wolverhampton, 2018
 ****************************************************************************/


int main() {

  cudaDeviceProp device_properties;
  cudaGetDeviceProperties(&device_properties, 0);

  printf("Maximum number of threads per block = %d\n",
          device_properties.maxThreadsPerBlock);

  printf("Maximum size of each dimension of a block = [%d][%d][%d]\n",
          device_properties.maxThreadsDim[0],
          device_properties.maxThreadsDim[1],
          device_properties.maxThreadsDim[2]);

  printf("Maximum size of each dimension of a grid = [%d][%d][%d]\n",
          device_properties.maxGridSize[0],
          device_properties.maxGridSize[1],
          device_properties.maxGridSize[2]);

  return 0;
}

Writing dq03.cu


In [ ]:
!nvcc -o dq03 dq03.cu

In [ ]:
!./dq03

Maximum number of threads per block = 1024
Maximum size of each dimension of a block = [1024][1024][64]
Maximum size of each dimension of a grid = [2147483647][65535][65535]


In [ ]:
%%writefile factorise_3_cude.cu
/****************************************************************************
  Similar to the factorise programs studied with POSIX threads, only this
  version runs on a GPU with CUDA.

  Compile with:
    nvcc -o factorise_3_cuda factorise_3_cuda.cu -lrt

  Dr Kevan Buckley, University of Wolverhampton, 2018
*****************************************************************************/

#include <stdlib.h>
#include <stdio.h>
#include <ctype.h>
#include <errno.h>
#include <sys/stat.h>
#include <string.h>
#include <time.h>
//#include <pthread.h>
#include <math.h>

#define goal 98931313

__global__ void factorise(){
  int a = threadIdx.x;
  int b = blockIdx.x;
  int c = blockIdx.y;

  if(a*b*c == goal){
     printf("solution is %d, %d, %d\n", a, b, c);
  }
}

int time_difference(struct timespec *start, struct timespec *finish,
                              long long int *difference) {
  long long int ds =  finish->tv_sec - start->tv_sec;
  long long int dn =  finish->tv_nsec - start->tv_nsec;

  if(dn < 0 ) {
    ds--;
    dn += 1000000000;
  }
  *difference = ds * 1000000000 + dn;
  return !(*difference > 0);
}

int main() {
  cudaError_t error;
  struct timespec start, finish;
  long long int time_elapsed;

  //clock_gettime(CLOCK_MONOTONIC, &start);

  dim3 gd(1000, 1000, 1);
  dim3 bd(1000, 1, 1);
  factorise<<<gd, bd>>>();

  cudaDeviceSynchronize();

  error = cudaGetLastError();

  if(error){
    fprintf(stderr, "Kernel launch returned %d %s\n",
      error, cudaGetErrorString(error));
    return 1;
  } else {
    fprintf(stderr, "Kernel launch successful.\n");
  }
  //clock_gettime(CLOCK_MONOTONIC, &finish);
  time_difference(&start, &finish, &time_elapsed);
  printf("Time elapsed was %lldns or %0.9lfs\n",
    time_elapsed, (time_elapsed/1.0e9));

  return 0;
}

Writing factorise_3_cude.cu


In [ ]:
!nvcc -o factorise_3_cude  factorise_3_cude.cu

In [ ]:
!./factorise_3_cude

solution is 997, 449, 221
solution is 449, 997, 221
solution is 997, 221, 449
solution is 221, 997, 449
solution is 449, 221, 997
solution is 221, 449, 997
Kernel launch successful.
Time elapsed was 0ns or 0.000000000s


In [ ]:
%%writefile 01.cu
#include <stdio.h>
#include <cuda_runtime_api.h>

/******************************************************************************
  Very simple CUDA program that shows the principles of copying data to and
  from a GPU and dynamic memory allocation on a GPU. The standard pattern for
  a lot of GPU work is:

    1) Prepare the data on the host part of the program. By host we mean the
       the CPU. In this case, the h_n integer is set to 19. The h_ prefix
       indicates a host variable, i.e. one that we will use with the CPU side
       of the program.
    2) Allocate memory on the device. By device we mean GPU. In this case a
       single integer, identified by d_n, is allocated using cudaMalloc. The
       d_ prefix indicates a device variable, i.e. one that we will use with
       the GPU side of the program.
    3) Transfer data from the host to device. In this case cudaMemcpy is used
       to copy the contents of h_n to d_n.
    4) The kernel function is invoked. In this case the kernel function is
       called kernel and is defined as __global__ which means a function
       that will execute on the device but is invoked from the host. The
       <<<1,1>>> part indicates that we want to execute the kernel with one
       thread block consisting of one thread. The kernel function here will
       only be invoked once in total.
    5) The kernel function is executed, which in this case sets the contents
       of the memory pointed to by d_n to 97.
    6) Data is copied from the device to the host. In this case the contents
       of memory pointed to by d_n are copied into the h_n variable.
    7) Dynamically allocated memory is freed using cudaFree.
    8) Results are output. In this case the value of h_n is printed, and if
       all goes well should print 97.

  CUDA functions return an integer code. If this code is not equal to zero
  something has gone wrong. cudaGetErrorString returns a description of an
  error given its code This program is rather paranoid and checks
  the return codes of all call CUDA function calls and terminates the program
  if zero was not returned.

  To compile:
    nvcc -o 01 01.cu

  Dr Kevan Buckley, University of Wolverhampton, 2018
******************************************************************************/

__global__ void kernel(int *n){
  *n = 97; // this is an arbitary number, just to see some results.
}

int main() {
  cudaError_t error;
  int *d_n;
  int h_n = 19;

  error = cudaMalloc(&d_n, sizeof(int));
  if(error){
    fprintf(stderr, "cudaMalloc on d_n returned %d %s\n", error,
      cudaGetErrorString(error));
    exit(1);
  }

  error = cudaMemcpy(d_n, &h_n, sizeof(int), cudaMemcpyHostToDevice);
  if(error){
    fprintf(stderr, "cudaMemcpy to d_n returned %d %s\n", error,
      cudaGetErrorString(error));
  }

  kernel <<<1,1>>>(d_n);
  cudaThreadSynchronize();

  error = cudaMemcpy(&h_n, d_n, sizeof(int), cudaMemcpyDeviceToHost);
  if(error){
    fprintf(stderr, "cudaMemcpy to h_n returned %d %s\n", error,
      cudaGetErrorString(error));
  }

  error = cudaFree(d_n);
  if(error){
    fprintf(stderr, "cudaFree on d_n returned %d %s\n", error,
      cudaGetErrorString(error));
    exit(1);
  }

  printf("result: h_n = %d\n", h_n);
  return 0;
}

Writing 01.cu


In [ ]:
!nvcc -o 01  01.cu

01.cu: In function ‘int main()’:
01.cu:68:22: warning: ‘cudaError_t cudaThreadSynchronize()’ is deprecated []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wdeprecated-declarations-Wdeprecated-declarations]8;;]
   68 |   cudaThreadSynchronize();
      |   ~~~~~~~~~~~~~~~~~~~^~
/usr/local/cuda/bin/../targets/x86_64-linux/include/cuda_runtime_api.h:1069:46: note: declared here
 1069 | extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
      |                                              ^~~~~~~~~~~~~~~~~~~~~


In [ ]:
!./01

result: h_n = 97


In [ ]:
%%writefile 01b.cu
#include <stdio.h>
#include <cuda_runtime_api.h>

/******************************************************************************
  Very simple CUDA program that shows the principles of copying data to and
  from a GPU and dynamic memory allocation on a GPU. The standard pattern for
  a lot of GPU work:

    1) Prepare the data on the host part of the program. By host we mean the
       the CPU. In this case, the h_n integer is set to 19. The h_ prefix
       indicates a host variable, i.e. one that we will use with the CPU side
       of the program.
    2) Allocate memory on the device. By device we mean GPU. In this case a
       single integer, identified by d_n, is allocated using cudaMalloc. The
       d_ prefix indicates a device variable, i.e. one that we will use with
       the GPU side of the program.
    3) Transfer data from the host to device. In this case cudaMemcpy is used
       to copy the contents of h_n to d_n.
    4) The kernel function is invoked. In this case the kernel function is
       called kernel and is defined as __global__ which means a function
       that will execute on the device but is invoked from the host. The
       <<<1,1>>> part indicates that we want to execute the kernel with one
       thread block consisting of one thread. The kernel function here will
       only be invoked once in total.
    5) The kernel function is executed, which in this case sets the contents
       of the memory pointed to by d_n to 97.
    6) Data is copied from the device to the host. In this case the contents
       of memory pointed to by d_n are copied into the h_n variable.
    7) Dynamically allocated memory is freed using cudaFree.
    8) Results are output. In this case the value of h_n is printed, and if
       all goes well should print 97.

  This version does no checking for errors. Its purpose is to show the main
  functionality. For the "real" version of the program error checking should be
  included. See 01.cu for this.

  To compile:
    nvcc 01b.cu

  Dr Kevan Buckley, University of Wolverhampton, 2018
******************************************************************************/

__global__ void kernel(int *n){
  *n = 97; // this is an arbitary number, just to see some results.
}

int main() {
  int *d_n;
  int h_n = 19;

  cudaMalloc(&d_n, sizeof(int));

  cudaMemcpy(d_n, &h_n, sizeof(int), cudaMemcpyHostToDevice);

  kernel <<<1,1>>>(d_n);

  cudaThreadSynchronize();

  cudaMemcpy(&h_n, d_n, sizeof(int), cudaMemcpyDeviceToHost);

  cudaFree(d_n);

  printf("result: h_n = %d\n", h_n);
  return 0;
}

Writing 01b.cu


In [ ]:
!nvcc -o 01b  01b.cu

01b.cu: In function ‘int main()’:
01b.cu:57:22: warning: ‘cudaError_t cudaThreadSynchronize()’ is deprecated []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wdeprecated-declarations-Wdeprecated-declarations]8;;]
   57 |   cudaThreadSynchronize();
      |   ~~~~~~~~~~~~~~~~~~~^~
/usr/local/cuda/bin/../targets/x86_64-linux/include/cuda_runtime_api.h:1069:46: note: declared here
 1069 | extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
      |                                              ^~~~~~~~~~~~~~~~~~~~~


In [ ]:
!./01b

result: h_n = 97


In [ ]:
%%writefile 02.cu
